#### 1. 기간 설정시 날짜 입력 안됨 문제

In [ ]:
import time
import pandas as pd
import re

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
# 크롤링 딜레이 몇초줄껀지
delay = 5

In [ ]:
driver = webdriver.Chrome('chromedriver')
url = 'https://search.naver.com/search.naver?query=&where=news&ie=utf8&sm=nws_hty'
driver.get(url)

In [ ]:
data_dic = {
     'date' : []
    ,'time' : []
    ,'title' : []
    ,'content' : []
    ,'category' : [] # 0이면 일반 1이면 사진
    ,'url' : []
}

In [ ]:
# 검색옵션 클릭
driver.find_element_by_css_selector('#_search_option_btn').click()

# 최신순 선택
driver.find_element_by_css_selector('#snb > div > ul > li:nth-child(3) > a').click()
driver.find_element_by_css_selector('#snb > div > ul > li.menu.selected > div > ul > li:nth-child(2) > a').click()

# 언론사 선택
driver.find_element_by_css_selector('#news_popup > a').click()
# 연합뉴스 체크
driver.find_element_by_css_selector('#ca_1001').click()
# 확인
driver.find_element_by_css_selector('#_nx_option_media > div.con_bx > div.view_btn > button.impact._submit_btn').click()

# # 날짜 설정
# start_date = '20180101'
# end_date = '2018.02.01'
# # 기간 클릭
# driver.find_element_by_css_selector('#snb > div > ul > li:nth-child(4) > a').click()
# # 날짜 입력
# #news_input_period_begin
# driver.find_element_by_css_selector('#news_input_period_begin').send_keys(start_date)
# # driver.find_element_by_css_selector('#news_input_period_end').send_keys(end_date)

In [ ]:
# 뉴스 키워드 입력
keyword = '날씨'
# 검색
driver.find_element_by_css_selector('#news_inner_search_query').send_keys(keyword)
driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div > div.news_srch > span > button').click()
# 최신순 정렬
driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.news_option > ul > li.select').click()

In [ ]:
# 전체 번호 세팅
total_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.title_desc.all_my > span').text
total_page = total_page.split('/')
total_page = total_page[1]
total_page = total_page.replace(" ", "")
total_page = total_page.replace(",", "")
total_page = total_page.replace("건", "")
total_page = int(total_page)

# 현재 번호
current_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.paging > strong').text
current_page = int(current_page)

In [ ]:
# 다 끝났으면 다음 페이지로 이동하는 모듈
def moveNextPage():
    global current_page
    current_page = current_page + 1
    # 페이징 태그 가져오기
    paging_elements = driver.find_elements_by_css_selector(".paging > a")
    for paging_num in paging_elements:
        if(paging_num.text == str(current_page)):
            paging_num.click()
            break

In [ ]:
def getContent():
    content = ''
    category = 0
    try:
        element = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#articleWrap > div.article")))
        content = driver.find_element_by_css_selector('#articleWrap > div.article').text
    except NoSuchElementException:
        print("[예외1]", driver.current_url)
        content = "[예외] 사진 추정"
        categoty = 1
    except TimeoutException:
        try:
            content = driver.find_element_by_css_selector('#viewWrap > div.inner-article > div.article-txt').text
            categoty = 1
        except NoSuchElementException:
            print("[예외2]", driver.current_url)
            content = "[예외] 뭘까"
            categoty = 1
        except TimeoutException:
            print("[예외3]", driver.current_url)
            content = "[예외] 으헝"
            categoty = 1
    except:
        print("[예외4] 저주받은 에러", driver.current_url)
    finally:
        return content, category

In [ ]:
def getDateTime():
    try:
        date_time = re.findall('\d+',driver.find_element_by_css_selector('#articleWrap > div.article-sns-md.sns-md04 > div.share-info > span > em').text)
    except:
        date_time = re.findall('\d+',driver.find_element_by_css_selector('#viewWrap > div.inner-article > div.article-copy > span.article-copy03').text)
    finally:
        return "".join(date_time)

In [ ]:
# 전체 시작
while(True):
#     if(current_page * 10 >= total_page):
    if(current_page * 10 >= 100):
        print(current_page*10,'크롤링 끝1')
        break
    
    page_content = driver.find_elements_by_css_selector('.type01 > li')
    for content in page_content:
        # 제목 담기
        data_dic['title'].append(content.find_element_by_css_selector('._sp_each_title').text)
        # URL 담기
        data_dic['url'].append(content.find_element_by_css_selector('._sp_each_title').get_attribute("href"))
        
        # 링크 클릭
        content.find_element_by_css_selector('._sp_each_title').click()
        # 최근 열린 탭으로 전환
        driver.switch_to.window(driver.window_handles[-1])
        ### 새로운 탭에서 작업 ###
        [content, category] = getContent()
        data_dic['content'].append(content)
        data_dic['category'].append(category)
        data_dic['date'].append(getDateTime()[0:8])
        data_dic['time'].append(getDateTime()[8:12])
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    moveNextPage()

In [ ]:
df = pd.DataFrame.from_dict(data_dic)
# df.to_csv('20152018날씨.csv', index=False, encoding='utf-8-sig')    
df.to_csv('test.csv', index=False, encoding='utf-8-sig')